In [22]:
import tensorflow as tf
import numpy as np
import os

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [23]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range = 15,
    horizontal_flip= True,
    width_shift_range=0.1,
    height_shift_range=0.1
)
datagen.fit(x_train)

x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
mean = np.mean(x_train)
std = np.std(x_train)
x_test = (x_test - mean) / std
x_train = (x_train - mean) / std
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [24]:
def resnet_block32(input_layer, filter, down_sample):
  x = input_layer
  for i in range(5):
    identity_layer = x
    x = tf.keras.layers.Conv2D(filters = filter, activation='relu', kernel_size = (3, 3), strides= (1, 1), padding='SAME', kernel_regularizer=tf.keras.regularizers.l2(0.0004), kernel_initializer=tf.keras.initializers.he_normal())(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    
    x = tf.keras.layers.Conv2D(filters = filter, activation='relu', kernel_size = (3, 3), strides= (1, 1), padding='SAME', kernel_regularizer=tf.keras.regularizers.l2(0.0004), kernel_initializer=tf.keras.initializers.he_normal())(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    
    identity_channel = identity_layer.shape.as_list()[-1]    
    print(identity_channel)
    if identity_channel != filter:
      identity_layer = tf.keras.layers.Conv2D(filter, kernel_size=(1, 1), strides=(1, 1), padding= 'SAME')(identity_layer) 
    x = tf.keras.layers.Add()([x, identity_layer])
  if down_sample == 0:
    x = tf.keras.layers.Conv2D(filter, kernel_size = (1, 1), strides=(2, 2), padding='same')(x)
  return x

In [25]:
x_input = tf.keras.layers.Input(shape = (32,32,3))
x = tf.keras.layers.Conv2D(filters = 16, kernel_size = (3, 3), strides= (1, 1), padding = 'SAME',kernel_regularizer=tf.keras.regularizers.l2(0.0004), kernel_initializer=tf.keras.initializers.he_normal())(x_input)
x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.Activation('relu')(x)

In [26]:
filters = 16
down_sample = [0, 0, 1]
for i in range(3):
  x = resnet_block32(x, filters, down_sample[i])
  filters = filters * 2
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(units = 10, activation = 'softmax', kernel_initializer=tf.keras.initializers.he_normal())(x)

model = tf.keras.Model(inputs=x_input, outputs = x)
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(lr=0.01, momentum=0.9),
    metrics=['accuracy'],
)
def lr_schedule(epoch):
  if epoch > 170 :
    return 0.0001
  if epoch > 130 :
    return 0.001
  if epoch > 70 :
    return 0.01
  return 0.03
  
reduced_lr = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

16
16
16
16
16
16
32
32
32
32
32
64
64
64
64


In [27]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_105 (Conv2D)            (None, 32, 32, 16)   448         ['input_4[0][0]']                
                                                                                                  
 batch_normalization_93 (BatchN  (None, 32, 32, 16)  64          ['conv2d_105[0][0]']             
 ormalization)                                                                                    
                                                                                                  
 activation_3 (Activation)      (None, 32, 32, 16)   0           ['batch_normalization_93[0]

In [28]:
model.fit(datagen.flow(x_train, y_train, batch_size=96), callbacks=[reduced_lr],
                       steps_per_epoch = len(x_train) // 96, epochs = 250, validation_data=(x_test, y_test), verbose = 1)

Epoch 1/250
520/520 [==============================] - 22s 38ms/step - loss: 2.3732 - accuracy: 0.4471 - val_loss: 2.1998 - val_accuracy: 0.5054 - lr: 0.0300
Epoch 2/250
520/520 [==============================] - 19s 37ms/step - loss: 1.8504 - accuracy: 0.5882 - val_loss: 1.7898 - val_accuracy: 0.5919 - lr: 0.0300
Epoch 3/250
520/520 [==============================] - 21s 39ms/step - loss: 1.5543 - accuracy: 0.6571 - val_loss: 1.4144 - val_accuracy: 0.7015 - lr: 0.0300
Epoch 4/250
520/520 [==============================] - 21s 40ms/step - loss: 1.3543 - accuracy: 0.7012 - val_loss: 1.2819 - val_accuracy: 0.7226 - lr: 0.0300
Epoch 5/250
520/520 [==============================] - 19s 37ms/step - loss: 1.2191 - accuracy: 0.7284 - val_loss: 1.2948 - val_accuracy: 0.7109 - lr: 0.0300
Epoch 6/250
520/520 [==============================] - 20s 38ms/step - loss: 1.1240 - accuracy: 0.7434 - val_loss: 1.0945 - val_accuracy: 0.7523 - lr: 0.0300
Epoch 7/250
520/520 [==============================]